## Introduction

Many problems in quantitative fields such as finance and engineering are optimization problems. Optimization problems lie at the core of complex decision-making and definition of strategies.

Optimization (or combinatorial optimization) means searching for an optimal solution in a finite or countably infinite set of potential solutions. Optimality is defined with respect to some criterion function, which is to be minimized or maximized. This is typically called cost function or objective function.

**Typical optimization problems**

Minimization: cost, distance, length of a traversal, weight, processing time, material, energy consumption, number of objects

Maximization: profit, value, output, return, yield, utility, efficiency, capacity, number of objects

We consider here max-cut problems of practical interest in many fields, and show how they can be mapped on quantum computers manually and how Qiskit optimization module supports this.


## Weighted Max-Cut

Max-Cut is an NP-complete problem, with applications in clustering, network science, and statistical physics. To grasp how practical applications are mapped into given Max-Cut instances, consider a system of many people that can interact and influence each other. Individuals can be represented by vertices of a graph, and their interactions seen as pairwise connections between vertices of the graph, or edges. With this representation in mind, it is easy to model typical marketing problems. For example, suppose that it is assumed that individuals will influence each other's buying decisions, and knowledge is given about how strong they will influence each other. The influence can be modeled by weights assigned on each edge of the graph. It is possible then to predict the outcome of a marketing strategy in which products are offered for free to some individuals, and then ask which is the optimal subset of individuals that should get the free products, in order to maximize revenues.

The formal definition of this problem is the following:

Consider an $n$-node undirected graph *G = (V, E)* where *|V| = n* with edge weights $w_{ij}>0$, $w_{ij}=w_{ji}$, for $(i, j)\in E$. A cut is defined as a partition of the original set V into two subsets. The cost function to be optimized is in this case the sum of weights of edges connecting points in the two different subsets, *crossing* the cut. By assigning $x_i=0$ or $x_i=1$ to each node $i$, one tries to maximize the global profit function (here and in the following summations run over indices 0,1,...n-1)

$$C(\textbf{x}) = \sum_{i \in V}\sum_{j \in V} w_{ij} x_i (1-x_j)$$

In our simple marketing model, $w_{ij}$ represents the probability that the person $j$ will buy a product after $i$ gets a free one. Note that the weights $w_{ij}$ can in principle be greater than $1$ (or even negative), corresponding to the case where the individual $j$ will buy more than one product. Maximizing the total buying probability corresponds to maximizing the total future revenues. In the case where the profit probability will be greater than the cost of the initial free samples, the strategy is a convenient one. An extension to this model has the nodes themselves carry weights, which can be regarded, in our marketing model, as the likelihood that a person granted with a free sample of the product will buy it again in the future, but for the moment let's stick with the simpler case.

In order to find a solution to this problem on a quantum computer, one needs first to map it to an Ising Hamiltonian. This can be done with the assignment $x_i\rightarrow (1-Z_i)/2$, where $Z_i$ is the Pauli Z operator that has eigenvalues $\pm 1$. Doing this we find that

$$C(\textbf{Z}) = \sum_{i,j} \frac{w_{ij}}{4} (1-Z_i)(1+Z_j) = -\frac{1}{2} \sum_{i<j} w_{ij} Z_i Z_j +\mathrm{const},$$

where $\mathrm{const} = \sum_{i<j}w_{ij}/2$, which can be ignored for max/min problems. In other terms, the weighted Max-Cut problem is equivalent to minimizing the Ising Hamiltonian

$$ H = \sum_{i<j} w_{ij} Z_iZ_j.$$

## Quantum Approximate Optimization Algorithm (QAOA)

The Quantum Approximate Optimization Algorithm (QAOA) is a hybrid quantum-classical algorithm designed to solve combinatorial optimization problems. It operates by encoding the problem into a cost Hamiltonian $H_P$, whose ground state represents the optimal solution. QAOA alternates between applying problem-specific and mixing Hamiltonians $H_M$, parameterized by angles, to evolve an initial quantum state toward a solution. These angles are optimized using a classical optimizer to maximize the probability of measuring a state corresponding to the best solution. QAOA is particularly well-suited for problems like Max-Cut, MIS, and others, offering a balance between quantum capabilities and classical optimization.

**Problem Input:**

1.	Objective Function:
An optimization problem is expressed as a cost function  $C(z)$ , where  $z \in \{0, 1\}^n$  represents the binary decision variables.

2.	Problem Hamiltonian:
The cost function is mapped to a diagonal Hamiltonian  $H_C$  in the computational basis:

$$H_P |z\rangle = C(z) |z\rangle, \quad \forall z \in \{0, 1\}^n.$$

3.	Mixing Hamiltonian:
A “mixing” Hamiltonian  $H_M$  is chosen to encourage exploration of the solution space. Typically:

$$H_M = \sum_{i=1}^n X_i,$$

where  $X_i$  is the Pauli-X operator applied to qubit  $i$ .

**Algorithm Parameters:**

1.	Initial State:
The algorithm begins with an easily prepared quantum state, usually the equal superposition:

$$|\psi_0\rangle = \frac{1}{\sqrt{2^n}} \sum_{z \in \{0, 1\}^n} |z\rangle.$$

2.	Parameter Angles:
Two sets of  $p$ -dimensional parameters are introduced:

$$\boldsymbol{\gamma} = (\gamma_1, \gamma_2, \ldots, \gamma_p), \quad \boldsymbol{\beta} = (\beta_1, \beta_2, \ldots, \beta_p).$$


**Quantum Circuit:**

The algorithm alternates between applying the cost Hamiltonian and mixer Hamiltonian over  $p$  layers, creating the parameterized state:

$$|\psi(\boldsymbol{\gamma}, \boldsymbol{\beta})\rangle = \prod_{k=1}^p \left( e^{-i \gamma_k H_P} e^{-i \beta_k H_M} \right) |\psi_0\rangle.$$


**Classical Optimization:**

The parameters  $\boldsymbol{\gamma}$  and  $\boldsymbol{\beta}$  are optimized using a classical algorithm to maximize the expected value of the problem Hamiltonian:

$$F(\boldsymbol{\gamma}, \boldsymbol{\beta}) = \langle \psi(\boldsymbol{\gamma}, \boldsymbol{\beta}) | H_P | \psi(\boldsymbol{\gamma}, \boldsymbol{\beta}) \rangle.$$

After optimization, the state  $|\psi(\boldsymbol{\gamma}^f, \boldsymbol{\beta}^f)\rangle$ is measured in the computational basis, yielding a candidate solution $z$. Repeated measurements yield samples that approximate the optimal solution.

*References:*

- \ A. Lucas, Frontiers in Physics 2, 5 (2014)
- \ E. Farhi, J. Goldstone, S. Gutmann, e-print arXiv 1411.4028 (2014)
- \ D. Wecker, M. B. Hastings, M. Troyer, Phys. Rev. A 94, 022309 (2016)
- \ E. Farhi, J. Goldstone, S. Gutmann, H. Neven, e-print arXiv 1703.06199 (2017)

# WEIGHTED MAX-CUT PROBLEM WITH QISKIT

Given an arbitary weighted undirected graph, implement the circuit that solves the corresponding Max-Cut problem on gate-based quantum machines.

In [ ]:
!git clone https://github.com/LINKS-Foundation-CPE/Hes-SO-quantum-course-2025.git

In [ ]:
%cd /content/Hes-SO-quantum-course-2025/

In [ ]:
!pip install qiskit==0.42.1
!pip install qiskit-aer==0.12.0

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
random.seed(42)
from qiskit.circuit import Parameter
from scipy.optimize import minimize
from qiskit.visualization import plot_histogram

from src.qiskit_qaoa.utils import *

In [ ]:
n = 5
graph = nx.gnp_random_graph(n, p=0.5, seed=3)
for (u,v,w) in graph.edges(data=True):
    w['weight'] = random.randint(1,5)
pos = nx.spring_layout(graph, seed=7)  # positions for all nodes - seed for reproducibility

# nodes
nx.draw_networkx_nodes(graph, pos, node_size=700)
# edges
nx.draw_networkx_edges(graph, pos, width=3)
# node labels
nx.draw_networkx_labels(graph, pos, font_size=20, font_family="sans-serif")
# edge weight labels
edge_labels = nx.get_edge_attributes(graph, "weight")
nx.draw_networkx_edge_labels(graph, pos, edge_labels)

ax = plt.gca()
plt.axis("off")
plt.tight_layout()
plt.show()

adjacency = nx.adjacency_matrix(graph).todense()

In [ ]:
# mixing Hamiltonian
beta = Parameter("beta")
qc_mix = QuantumCircuit(n)
for i in range(n):
    qc_mix.rx(beta, i)

In [ ]:
# problem Hamiltonian
gamma = Parameter("gamma")
qc_p = QuantumCircuit(n)
for pair in list(graph.edges()):  # pairs of nodes
    qc_p.rzz(gamma, pair[0], pair[1])

In [ ]:
# state initialization
qc_0 = QuantumCircuit(n)
for i in range(n):
    qc_0.h(i)

In [ ]:
qc_qaoa = QuantumCircuit(n)
qc_qaoa.append(qc_0, range(n))
qc_qaoa.append(qc_p, range(n))
qc_qaoa.append(qc_mix, range(n))
qc_qaoa.decompose().decompose().draw()

In [ ]:
expectation = get_expectation(graph)
res = minimize(expectation,
               x0=[0,0],
               method='Powell')
res

In [ ]:
qc_res = create_qaoa_circ(graph, res.x)
counts = AerSimulator().run(qc_res, seed_simulator=10, shots=100).result().get_counts()
plot_histogram(counts, sort='value_desc')

In [ ]:
for sol in sorted(counts, key=counts.get, reverse=True):
    vertexes = [idx for idx, item in enumerate(sol) if '1' in item]
    print(f'Vertexes: {vertexes} --- Cut: { -maxcut_obj(sol, graph)}')